In [61]:
import requests as requests
import json
import datetime
from datetime import date

keys={}
with open("/Users/jagluck/Documents/GitHub/new-artists-twitter-bot/keys.json","r") as f:
    keys = json.loads(f.read())
    
# Consumer keys and access tokens, used for OAuth
songkick_api_key = keys['songkick_api_key']
dc_metro_id = "1409"

In [125]:
def getArtistId(artist):
    url = "https://api.songkick.com/api/3.0/search/artists.json"
    params = {'apikey': songkick_api_key, "query" : artist}
    resp = requests.get(url, params=params) 
    return resp.text

def getUpcomingShowsPage(minDate,MaxDate,page):
    url = "https://api.songkick.com/api/3.0/metro_areas/" + dc_metro_id + "/calendar.json"
    params = {'apikey': songkick_api_key, "min_date" : minDate, "max_date" : maxDate, "page" : page, "per_page" : 50}
    resp = requests.get(url, params=params) 
    return resp.text

def getUpcomingShows(daysAhead):
    page = 1
    minDate = str(date.today())
    maxDate = str(date.today() + datetime.timedelta(daysAhead))
    resp = getUpcomingShowsPage(minDate,maxDate, page)
    
    eventIds = []
    eventTypes = []
    eventUrls = []
    dates = []
    venueIds = []
    venueNames = []
    locationCities = []

    for event in events:
        eventIds.append(event['id']) 
        eventTypes.append(event['type'])
        eventUrls.append(event['uri'])
        dates.append(event['start']['date'])
        venueIds.append(event['venue']['id'])
        venueNames.append(event['venue']['displayName'])
        locationCities.append(event['location']['city']) 
    
    data = pd.DataFrame(
            {
                "eventId" : eventIds,
                "eventType" : eventTypes,
                "eventUrl" : eventUrls,
                "date" : dates,
                "venueId" : venueIds,
                "venueName" : venueNames,
                "locationCity" : locationCities
            })
    
    return data

In [97]:
artist = "blink 182"
resp = getArtistKey(artist)

In [126]:
data = getUpcomingShows(7)

NameError: name 'pd' is not defined